In [1]:
#Importing all the required libraries
import cv2
import mediapipe as mp
import pyautogui
import time

In [2]:
# Initializing MediaPipe Hand model

mp_hands=mp.solutions.hands
mp_drawings=mp.solutions.drawing_utils
hands=mp_hands.Hands(max_num_hands=1)



In [10]:
video=cv2.VideoCapture(0)
prev=-1
while True:
    suc,frame=video.read()
    if not suc:
        break
    frame=cv2.flip(frame,1)
    Forward='Forward : 1'
    Backward='Backward : 2'
    Audioup='Audio Up : 3'
    Audiodown='Audio Down : 4'
    playpause='Play/Pause : 5'
    cv2.putText(frame,Forward,(20,20),cv2.FONT_HERSHEY_COMPLEX_SMALL,0.8,(0,0,0),2)
    cv2.putText(frame,Backward,(20,40),cv2.FONT_HERSHEY_COMPLEX_SMALL,0.8,(0,0,0),2)
    cv2.putText(frame,Audioup,(20,60),cv2.FONT_HERSHEY_COMPLEX_SMALL,0.8,(0,0,0),2)
    cv2.putText(frame,Audiodown,(20,80),cv2.FONT_HERSHEY_COMPLEX_SMALL,0.8,(0,0,0),2)
    cv2.putText(frame,playpause,(20,100),cv2.FONT_HERSHEY_COMPLEX_SMALL,0.8,(0,0,0),2)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    
    result = hands.process(rgb_frame)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    tipids=[4,8,12,16,20]
    lmlist=[]
    if result.multi_hand_landmarks:
        for landmark in result.multi_hand_landmarks:
            for id,lm in enumerate(landmark.landmark):
                    cx=lm.x
                    cy=lm.y
                    lmlist.append([id,cx,cy])
                    if len(lmlist)!=0 and len(lmlist)==21:
                        fingerlist=[]
                        #thumb
                        if lmlist[8][1]<lmlist[20][1]:
                            if lmlist[4][1]>lmlist[3][1]:
                                fingerlist.append(0)
                            else:
                                fingerlist.append(1)
                        if lmlist[8][1]>lmlist[20][1]:
                            if lmlist[4][1]<lmlist[3][1]:
                                fingerlist.append(0)
                            else:
                                fingerlist.append(1)
                        #fingers                               
                        for i in range(1,5):
                            if lmlist[tipids[i]][2]>lmlist[tipids[i]-2][2]:
                                fingerlist.append(0)
                            else:
                                fingerlist.append(1)

            if len(fingerlist)!=0:
                    
                    fingercount=fingerlist.count(1)
            

                         
            if prev!=fingercount:           
                    time.sleep(0.2)
                    fingercount=fingerlist.count(1)
                    if fingercount==1:
                        pyautogui.press("right")
                        action='Forward'
                    if fingercount==2:
                        pyautogui.press("left")
                        action='Backward'
                    if fingercount==3:
                        pyautogui.press("volumeup")
                        action='Volume Up'
                    if fingercount==4:
                        pyautogui.press("volumedown")
                        action='Volume Down'
                    if fingercount==5:
                        pyautogui.press("space")
                        action='Play/Pause'
                    if fingercount==0:
                        action='base'
                
   
                    prev=fingercount

        cv2.putText(frame,action,(35,435),cv2.FONT_HERSHEY_COMPLEX,2,(255,255,0),2)
        mp_drawings.draw_landmarks(frame,landmark,mp_hands.HAND_CONNECTIONS)



    cv2.imshow("Window",frame)
    if cv2.waitKey(1) & 0XFF==ord("q"):
        break
video.release()
cv2.destroyAllWindows()
